In [1]:
from dotenv import load_dotenv
import os
import requests
load_dotenv()

def get_aqi_str(aqi_key:str)->str:
    url = f'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key={aqi_key}&limit=1000&sort=datacreationdate desc&format=JSON'
    try:
        r = requests.get(url)
        r.raise_for_status()   #這樣就不用檢查 200
    except Exception as e:
        print(f'發生錯誤: {e}')

    return r.text


aqi_data:str = get_aqi_str(aqi_key=os.environ["AQI_KEY"])

In [ ]:
from pydantic import BaseModel,Field

class Record(BaseModel):
    站點: str = Field(alias='sitename')
    縣市: str = Field(alias="county")
    aqi:int
    狀態: str = Field(alias='status')
    PM25: float = Field(alias='pm2.5')
    緯度: float = Field(alias='latitude')
    經度: float = Field(alias="longitude")





class AQI(BaseModel):
    records:list[Record]

aqi = AQI.model_validate_json(aqi_data)



In [ ]:
#驗證錯誤，pm2.5有空字串，無法轉換為float

In [9]:
from pydantic import BaseModel,Field,field_validator

class Record(BaseModel):
    站點: str = Field(alias='sitename')
    縣市: str = Field(alias="county")
    aqi: float | None
    狀態: str = Field(alias='status')
    PM25: float | None = Field(alias='pm2.5')
    緯度: float = Field(alias='latitude')
    經度: float = Field(alias="longitude")

    @field_validator('PM25','aqi',mode='before')   #先做下面的手動轉換 在做上面的自動轉換
    @classmethod
    def pm25_for_empty_string(cls,v:str)->float:
        if v =="":
            return None
        else:
            return float(v)



class AQI(BaseModel):
    records:list[Record]


aqi = AQI.model_validate_json(aqi_data)

In [ ]:
aqi.records

In [ ]:
aqi.model_dump()

In [ ]:
aqi.model_dump()['records']